In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
data = pd.read_csv("../input/turkish-employee-dataset/calisan_data.csv")
data = data[['Departman','DogumTarihi', 'Maas']]
dataset_year = 2019 #dataset tarihi
tecrube_baslangic_yasi = 21 #21 yaşından beri aynı sektörde çalıştığı farzediliyor.

In [2]:
for index, row in data.iterrows():
    data['DogumTarihi'][index] = (row['DogumTarihi'].split(" ")[2])
#datasetten doğum ay ve günü silindi.
for index, row in data.iterrows():
    data['Maas'][index] = int(row['Maas']) - int(row['Maas']%1000)
#Integer dönüşümü
data['MaxTecrube'] = pd.Series()
for index, row in data.iterrows():
    data['MaxTecrube'][index] = (dataset_year - tecrube_baslangic_yasi - int(row['DogumTarihi']))
#Varsayımsal olarak max tecrübe oluşturduk.
data = data[['Departman','MaxTecrube','Maas']]
data

,Departman,MaxTecrube,Maas
0,Müşteri İlişkileri ve Satış,13.0,9000
1,Yazılım Geliştirme,19.0,11000
2,Yazılım Geliştirme,19.0,11000
3,Yazılım Geliştirme,11.0,5000
4,Yazılım Geliştirme,11.0,8000
...,...,...,...
2440,Yazılım Geliştirme,23.0,12000
2441,Yazılım Geliştirme,12.0,6000
2442,Yazılım Geliştirme,15.0,8000
2443,Yazılım Geliştirme,14.0,9000


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Departman'] = le.fit_transform(data['Departman'])

In [4]:
data

,Departman,MaxTecrube,Maas
0,1,13.0,9000
1,2,19.0,11000
2,2,19.0,11000
3,2,11.0,5000
4,2,11.0,8000
...,...,...,...
2440,2,23.0,12000
2441,2,12.0,6000
2442,2,15.0,8000
2443,2,14.0,9000


In [5]:
from sklearn.model_selection import train_test_split
training_set, validation_set = train_test_split(data, test_size = 0.2, random_state = 21)
X_train = training_set.iloc[:,0:-1].values
Y_train = training_set.iloc[:,-1].values
X_val = validation_set.iloc[:,0:-1].values
y_val = validation_set.iloc[:,-1].values

In [6]:
def accuracy(confusion_matrix):
    carpraz_top = confusion_matrix.trace()
    el_top = confusion_matrix.sum()
    return float(carpraz_top) / float(el_top)

In [7]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, Y_train)

GaussianNB()

In [8]:
y_pred = classifier.predict(X_val)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

print("VERIM = ", accuracy(cm))

VERIM =  0.25357873210633947


In [9]:
Ornek = [
    [2,2], #2 yıllık tecrübesi olan yazılım geliştiricisi
    [2,6], #6 yıllık tecrübesi olan yazılım geliştiricisi
    [1,5], #5 yıllık tecrübesi olan müşteri ilişkileri çalışanı
    [2,19], #19 yıllık tecrübesi olan yazılım geliştiricisi
]
tahminler = classifier.predict(Ornek)
tahminler

array([ 3000,  5000,  4000, 11000])